In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Reading the raw dataset.

In [ ]:
df = pd.read_csv(open("../raw/parliementary_debates_2024.tsv"), sep="\t", encoding="utf-8")
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["id", "speaker", "sex", "text_en"])

Renaming columns.

In [ ]:
df = df.rename(columns={"text": "body", "label": "leaning"})

Categorizing the leaning column.

In [ ]:
df["leaning"].unique()

In [ ]:
df["leaning"] = df["leaning"].replace({0: "left", 1: "right"})
df["leaning"] = df["leaning"].astype("category")

Stripping the bodies.

In [ ]:
df["body"] = (df["body"].str.strip()).replace("", np.nan)

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Removing the HTML paragraph tags from the bodies.

In [ ]:
df["body"] = df["body"].str.replace(" <p>", "")

Inspecting the body length.

In [ ]:
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

In [ ]:
df["body_length"].mean()

The leaning distribution.

In [ ]:
df.groupby("leaning", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per leaning.

In [ ]:
df.groupby("leaning", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

The body length distribution by leaning.

In [ ]:
plt.figure(figsize=(10, 6))

for leaning in df["leaning"].unique():
    df_leaning = df[df["leaning"] == leaning]
    downsampled = pd.concat(
        # Ensuring to include the last (longest) item.
        [df_leaning["body_length"].iloc[::100], df_leaning["body_length"].tail(1)]
    ).drop_duplicates().reset_index(drop=True)
    plt.plot(downsampled, label=leaning)

plt.xlabel("downsampled index")
plt.ylabel("body length")
plt.title("body length by political leaning")
plt.legend(title="leaning")
plt.show()

In [ ]:
df.to_parquet("../preprocessed/parliament_speeches_2024.parquet")